<a href="https://colab.research.google.com/github/msmsm104/MachineLearning_Project/blob/main/Training_Model/Modeling_2%EC%B0%A8_0610.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML_Project(Modeling_2)

     => 팀원들간 파악한 인사이트와 생각을 기반으로 전처리한 데이터(2차)와 간단한 전처리 과정을 거친 데이터(1차)를 통해 모델링을 진행. 
     => 결과값을 비교

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/이어드림스쿨/TIL(Today I Learned)/20220607/ML-project/data/'

#### train data set의 row가 400만개 이상이므로 Kaggle에서 제공하는 코드를 통해 data의 memory usage를 줄인 다음 modeling 작업을 진행한다. (그렇지 않는다면 중간에 kernel이 끊기는 문제가 발생할 수 있다.)
#### 좀 더 코드를 간소화하기 위해 train_test_split과 modeling과정을 함수로 정의하고 적용.

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
train_1 = reduce_mem_usage(pd.read_csv(base_path + 'featured_train_1.csv')) ## 1차 전처리 데이터
train_2 = reduce_mem_usage(pd.read_csv(base_path + 'featured_train_2.csv')) ## 2차 전처리 데이터

Memory usage of dataframe is 712.48 MB
Memory usage after optimization is: 127.23 MB
Decreased by 82.1%
Memory usage of dataframe is 610.70 MB
Memory usage after optimization is: 106.02 MB
Decreased by 82.6%


In [ ]:
from sklearn.model_selection import train_test_split

def check_error(model_, data_):

    X = data_.drop(columns = 'winPlacePerc')
    y = data_['winPlacePerc']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

    model = model_
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return mean_absolute_error(y_test, pred)

### Installing CatBoost

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


## Modeling

    - LinearRegression
    - Lasso
    - Ridge
    - SVR(x)
    - KNeighborsRegressor(x)
    - DecisionTreeRegressor(x)
    - LGBMRegressor
    - RandomForestRegressor
    - XGBRegressor
    - CatBoostRegressor(x)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor

from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error

## Linear Reg, Lasso, Ridge, LGBMRegressor, RandomFor, XGBReg, CatBoost

### Linear Regression

In [ ]:
result_1 = check_error(LinearRegression(), train_1)
result_2 = check_error(LinearRegression(), train_2)
print(f"MAE of LinearRegression_1: {result_1:.4f}")
print(f"MAE of LinearRegression_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

MAE of LinearRegression_1: 0.0968
MAE of LinearRegression_2: 0.1049
Difference : 0.0081


### Lasso

In [ ]:
result_1 = check_error(Lasso(), train_1)
result_2 = check_error(Lasso(), train_2)
print(f"MAE of Lasso_1: {result_1:.4f}")
print(f"MAE of Lasso_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

MAE of Lasso_1: 0.1261
MAE of Lasso_2: 0.1261
Difference : 0.0000


### Ridge

In [ ]:
result_1 = check_error(Ridge(), train_1)
result_2 = check_error(Ridge(), train_2)
print(f"MAE of Ridge_1: {result_1:.4f}")
print(f"MAE of Ridge_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.82674e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


MAE of Ridge_1: 0.0968
MAE of Ridge_2: 0.1049
Difference : 0.0081


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.87965e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


### lightGBM

In [ ]:
result_1 = check_error(LGBMRegressor(), train_1)
result_2 = check_error(LGBMRegressor(), train_2)
print(f"MAE of LGBMRegressor_1: {result_1:.4f}")
print(f"MAE of LGBMRegressor_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

MAE of LGBMRegressor_1: 0.0638
MAE of LGBMRegressor_2: 0.0681
Difference : 0.0043


### XGBoost

In [ ]:
result_1 = check_error(XGBRegressor(), train_1)
result_2 = check_error(XGBRegressor(), train_2)
print(f"MAE of XGBRegressor_1: {result_1:.4f}")
print(f"MAE of XGBRegressor_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

[06:36:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:45:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE of XGBRegressor_1: 0.0721
MAE of XGBRegressor_2: 0.0741
Difference : 0.0020


### CatBoost

In [ ]:
result_1 = check_error(CatBoostRegressor(), train_1)
result_2 = check_error(CatBoostRegressor(), train_2)
print(f"MAE of CatBoostRegressor_1: {result_1:.4f}")
print(f"MAE of CatBoostRegressor_2: {result_2:.4f}")
print(f"Difference : {result_2 - result_1:.4f}")

Learning rate set to 0.149026
0:	learn: 0.2709265	total: 1.01s	remaining: 16m 48s
1:	learn: 0.2408531	total: 1.75s	remaining: 14m 31s
2:	learn: 0.2161753	total: 2.45s	remaining: 13m 34s
3:	learn: 0.1956704	total: 3.18s	remaining: 13m 11s
4:	learn: 0.1781773	total: 4.07s	remaining: 13m 29s
5:	learn: 0.1639876	total: 5.06s	remaining: 13m 58s
6:	learn: 0.1528903	total: 6.04s	remaining: 14m 17s
7:	learn: 0.1437100	total: 7.04s	remaining: 14m 32s
8:	learn: 0.1360397	total: 8.11s	remaining: 14m 53s
9:	learn: 0.1298822	total: 9.25s	remaining: 15m 15s
10:	learn: 0.1250981	total: 10.3s	remaining: 15m 24s
11:	learn: 0.1212832	total: 11.6s	remaining: 15m 53s
12:	learn: 0.1183214	total: 12.2s	remaining: 15m 28s
13:	learn: 0.1159840	total: 12.8s	remaining: 15m 1s
14:	learn: 0.1141425	total: 13.4s	remaining: 14m 42s
15:	learn: 0.1119187	total: 14.1s	remaining: 14m 28s
16:	learn: 0.1105299	total: 14.7s	remaining: 14m 12s
17:	learn: 0.1093203	total: 15.4s	remaining: 13m 59s
18:	learn: 0.1079654	total:

KeyboardInterrupt: ignored

In [ ]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
mean_absolute_error(y_test, pred)

Learning rate set to 0.149026
0:	learn: 0.2709777	total: 986ms	remaining: 16m 24s
1:	learn: 0.2407417	total: 1.71s	remaining: 14m 11s
2:	learn: 0.2157524	total: 2.5s	remaining: 13m 52s
3:	learn: 0.1952294	total: 3.35s	remaining: 13m 54s
4:	learn: 0.1786963	total: 4.14s	remaining: 13m 44s
5:	learn: 0.1651402	total: 4.88s	remaining: 13m 28s
6:	learn: 0.1531849	total: 5.64s	remaining: 13m 20s
7:	learn: 0.1442748	total: 6.38s	remaining: 13m 11s
8:	learn: 0.1365962	total: 7.16s	remaining: 13m 8s
9:	learn: 0.1309023	total: 7.88s	remaining: 13m
10:	learn: 0.1261145	total: 8.65s	remaining: 12m 58s
11:	learn: 0.1219894	total: 9.45s	remaining: 12m 58s
12:	learn: 0.1186175	total: 10.2s	remaining: 12m 52s
13:	learn: 0.1161783	total: 10.8s	remaining: 12m 44s
14:	learn: 0.1138015	total: 11.6s	remaining: 12m 42s
15:	learn: 0.1121928	total: 12.3s	remaining: 12m 35s
16:	learn: 0.1102336	total: 13.1s	remaining: 12m 34s
17:	learn: 0.1091108	total: 13.7s	remaining: 12m 28s
18:	learn: 0.1079360	total: 14.4

0.06197040020453851

---

## 1차, 2차 전처리 데이터 분석.

    => Lasso를 제외한 모든 모델에서 MAE 값이 모두 살짝씩 증가함을 알 수 있다. (오히려 모델의 성능이 낮아짐.)
    => 전처리가 이루어진 column들이 target과 상관관계가 낮은 column임을 가만하면 상관관계가 높은 column에 대한 새로운 인사이트 도출과 그에 따른 전처리과정이 필요하다고 판단된다.

    => 6/13까지 새로운 인사이트 도출을 위한 EDA, 전처리
    => 각 모델별 성능 변화를 데이터프레임으로 만들어서 보여줘도 이쁠거 같다 (해보고 싶은 분들은 시도 해도 좋을거 같습니다 => 성공하시면 자랑해주세요)